# Лабораторная работа 3.
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы БФИ2302 Рыбаков А.Д.
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом.

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса.

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



In [10]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import time
import heapq
from IPython.display import display


In [11]:
#Алгоритм Флойда-Уоршелла
def floyd_warshall(adj_matrix):
    num_nodes = len(adj_matrix)
    dist = adj_matrix.copy()
    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                if dist[i][k] and dist[k][j]:
                    new_distance = dist[i][k] + dist[k][j]
                    if dist[i][j] == 0 or new_distance < dist[i][j]:
                        dist[i][j] = new_distance
    return dist

In [12]:
#Алгоритм Дейкстры
def dijkstra(adj_matrix, start):
    num_nodes = len(adj_matrix)
    distances = {node: float('inf') for node in range(num_nodes)}
    distances[start] = 0
    priority_queue = [(0, start)]
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        if current_distance > distances[current_node]:
            continue
        for neighbor in range(num_nodes):
            weight = adj_matrix[current_node][neighbor]
            if weight > 0:
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    heapq.heappush(priority_queue, (distance, neighbor))
    return distances

In [13]:
#Алгоритм Беллмана-Форда
def bellman_ford_fixed(adj_matrix, start):
    num_nodes = len(adj_matrix)
    distances = {node: float('inf') for node in range(num_nodes)}
    distances[start] = 0
    for _ in range(num_nodes - 1):
        for u in range(num_nodes):
            for v in range(num_nodes):
                weight = adj_matrix[u][v]
                if weight > 0 and distances[u] != float('inf') and distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight
    return {k: (v if v != float('inf') else 0) for k, v in distances.items()}

In [14]:
#Алгоритм Джонсона
def johnson_fixed(adj_matrix):
    num_nodes = len(adj_matrix)
    new_matrix = np.zeros((num_nodes + 1, num_nodes + 1))
    new_matrix[:num_nodes, :num_nodes] = adj_matrix
    for i in range(num_nodes):
        new_matrix[num_nodes, i] = 0
    h = bellman_ford_fixed(new_matrix, num_nodes)
    new_adj_matrix = adj_matrix.copy()
    for u in range(num_nodes):
        for v in range(num_nodes):
            if new_adj_matrix[u, v] > 0 and u in h and v in h:
                new_adj_matrix[u, v] += h[u] - h[v]
    distances = np.zeros((num_nodes, num_nodes))
    for u in range(num_nodes):
        distances[u] = list(dijkstra(new_adj_matrix, u).values())
    return distances

In [15]:
#Алгоритм Левита
def levit(adj_matrix, start):
    num_nodes = len(adj_matrix)
    distances = {i: float('inf') for i in range(num_nodes)}
    distances[start] = 0
    main_queue = []
    urgent_queue = []
    main_queue.append(start)
    while main_queue or urgent_queue:
        if urgent_queue:
            u = urgent_queue.pop(0)
        else:
            u = main_queue.pop(0)
        for v in range(num_nodes):
            weight = adj_matrix[u][v]
            if weight > 0:
                if distances[v] == float('inf'):
                    distances[v] = distances[u] + weight
                    main_queue.append(v)
                elif distances[v] > distances[u] + weight:
                    distances[v] = distances[u] + weight
                    urgent_queue.append(v)
    return distances

In [16]:
#Алгоритм Йена
import heapq

def yen_k_shortest_paths(graph, start, end, k=3):
    def dijkstra_modified(g, src, dst, forbidden_edges):
        n = len(g)
        dist = [float('inf')] * n
        dist[src] = 0
        prev = [-1] * n
        heap = [(0, src)]
        
        while heap:
            d, u = heapq.heappop(heap)
            if u == dst:
                break
            if d > dist[u]:
                continue
            for v in range(n):
                if g[u][v] > 0 and (u, v) not in forbidden_edges:
                    if dist[v] > dist[u] + g[u][v]:
                        dist[v] = dist[u] + g[u][v]
                        prev[v] = u
                        heapq.heappush(heap, (dist[v], v))
        
        if dist[dst] == float('inf'):
            return None
        path = []
        u = dst
        while u != -1:
            path.append(u)
            u = prev[u]
        return path[::-1]

    # Первый путь
    A = [dijkstra_modified(graph, start, end, set())]
    if not A[0]:
        return []

    B = []  # Куча для кандидатов

    for _ in range(1, k):
        for i in range(len(A[-1]) - 1):
            spur_node = A[-1][i]
            root_path = A[-1][:i+1]
            
            # Запрещаем рёбра из предыдущих путей
            banned_edges = set()
            for p in A:
                if len(p) > i and p[:i+1] == root_path and i+1 < len(p):
                    banned_edges.add((p[i], p[i+1]))
            
            spur_path = dijkstra_modified(graph, spur_node, end, banned_edges)
            if spur_path:
                full_path = root_path[:-1] + spur_path
                cost = sum(graph[full_path[j]][full_path[j+1]] for j in range(len(full_path)-1))
                heapq.heappush(B, (cost, full_path))
        
        if not B:
            break
        A.append(heapq.heappop(B)[1])

    return A[:k]

In [17]:
#Анализ производительности
def analyze_final_corrected_algorithms(adj_matrix):
    num_nodes = len(adj_matrix)
    algorithms = {
        'Флойда-Уоршелла': floyd_warshall,
        'Дейкстры': lambda mat: dijkstra(mat, 0),
        'Беллмана-Форда': lambda mat: bellman_ford_fixed(mat, 0),
        'Джонсона': johnson_fixed,
        'Левита': lambda mat: levit(mat, 0),
        'Йена': lambda mat: yen_k_shortest_paths(mat, 0, len(mat), 3)
    }
    results = []
    for algo_name, algo_func in algorithms.items():
        start_time = time.perf_counter_ns()
        elapsed_time = time.perf_counter_ns() - start_time
        results.append((algo_name, num_nodes, elapsed_time))
    df = pd.DataFrame(results, columns=['Алгоритм', 'Ноды', 'Время (ns)'])
    return df

In [18]:
#Определение матрицы смежности тестов
with open("graphLab3.txt", "r") as file:
    source = file.read().strip().split("\n")
    test_adj_matrix = []
    for line in source:
        a = [int(x) for x in line.split(" ")]
        test_adj_matrix.append(a)
showed_test_adj_matrix = test_adj_matrix # backup for showing in pyvis

print("Test matrix")
print(*test_adj_matrix, sep="\n", end="\n")

test_adj_matrix = np.array(test_adj_matrix)


Test matrix
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 3, 0, 4, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 4, 0, 5, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 5, 0, 6, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 6, 0, 7, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 7, 0, 8, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 8, 0, 9, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0]


In [19]:
df_final_corrected_analysis = analyze_final_corrected_algorithms(test_adj_matrix)
display(df_final_corrected_analysis)

from pyvis import network as net
import os

g = net.Network(notebook=True, bgcolor="#FFFFFF", cdn_resources="remote")
g.add_nodes(
    list(range(len(showed_test_adj_matrix))), 
    title=[str(x) for x in range(len(showed_test_adj_matrix))],
    label=[str(x) for x in range(len(showed_test_adj_matrix))],
    color=["#171717" for _ in range(len(showed_test_adj_matrix))]
)

for a, line in enumerate(showed_test_adj_matrix):
    for b, dist in enumerate(line):
        if a==b: continue
        if dist: g.add_edge(a, b, weight=dist)

g.show("graph.html")
os.system("graph.html")


,Алгоритм,Ноды,Время (ns)
0,Флойда-Уоршелла,11,800
1,Дейкстры,11,200
2,Беллмана-Форда,11,100
3,Джонсона,11,100
4,Левита,11,100
5,Йена,11,200


graph.html


0

### Выполнение:

### Вывод